# Généralités

## Problématique liée à l'article 
Problématique spécifque : Débruitage d'images.

Problématique globale : Comprendre ce qui fait par essence, qu'une image est une image réelle et qu'une autre n'est pas une image réelle. Car mathématiquement une image est juste un vecteur à N composantes. Pourquoi un vecteur est interprété par notre cerveau comme une image réelle, alors qu'une autre ne l'est pas. (notion de prior). À quoi doit ressembler une image pour être une image ?

## Quel est l'objectif de l'article ?

Pour extraire une information d'un signal bruité, il existe des méthodes en traitement du signal de processus, de déduction basée sur des informations partielles du signal d'origine.

Ces méthodes ne sont pas efficaces dans le cas de traitement d'image, car le jeu de données est bien trop grand.

Les méthodes d'apprentissage automatique ont révolutionné ce problème de traitement d'image, il est maintenant facile par ces méthodes, comme le CNN, d'extraire une image originel d'une image bruitée.

Le problème de ces méthodes est qu'elles sont spécifiques à chaque problème. Les méthodes avec a priori (informations partielles) sont plus universels.

**L'objectif de cet article et d'essayer d'extraire d'un apprentissage automatique la manière dont il intègre de manière implicite une connaissance a priori sophistiqué des images. L'article s'intéresse particulièrement au cas d'un bruitage gaussien.**

### Cadre plus précis par rapport aux méthodes utlisées jusque là dans la littérature
Les deux méthodes de débruitage sont :
1. **Approche traditionnelle :**
$ \hat{x}(y) = \int x p(x|y)dx = \int x p(y|x)p(x)dx / p(y) $
   - Choix d'un modèle de probabilité a priori \(p(x)\).
   - Combinaison avec une fonction de vraisemblance \(p(y|x)\) décrivant le processus de mesure bruité.
   - Résolution du problème en calculant l'estimation du minimum d'erreur quadratique moyenne (MMSE) \(\hat{x}(y)\) à partir de l'intégration de la densité postérieure.
Problème : probabilité a priori \(p(x)\) difficile à obtenir voire impossible surotut dans le cas d'image.

2. **Approche moderne de débruitage par apprentissage supervisé :**
   - Apprentissage supervisé d'une fonction de mapping directe des images bruitées aux images débruitées à l'aide de réseaux de neurones convolutionnels (CNN).
   - Paramétrage de la fonction d'estimation en forme paramétrique.
   - Ajustement des paramètres en minimisant l'erreur quadratique moyenne du débruitage sur un ensemble d'entraînement étendu.
   - Les CNN, formés pour gérer divers niveaux de bruit, effectuent une projection approximative sur un sous-espace de basse dimension, interprété comme un hyperplan tangent de la variété d'images.
Problème : besoin d'un entraînement supervisé préalable qui amène souvent à un overfitting des paramètres, non universel.


### Obejctif final 
1. On prend une image
2. On lui applique un filtre gaussien
3. On la fait passer dans l'algorhitme pour retrouver une image la plus proche de l'image initiale

## Représentation des images réelles et bruitées

Nous pouvons voir les images comme des objets de l'ensemble des réels est noté $\mathbb{R}^N$. 
C'est la représentation que nous utilisons pour coder les images.

Nous pouvons représenter visuellement ces images sur un espace 2D pour mieux comprendre.
On peut voir le plan comme l'ensemble des images possibles $E$ et un point de ce plan correspond à une image $x\in E$.

- Les images visuelles/réelles/priors résident sur un sous espace de plus basse dimension que $E$. Les images sur ce sous espace sont approximativement équiprobables - du moins localement. La probabilité $p(x)$ que $x$ soit une image naturelle est nulle partout sauf là où $x$ réside sur l'espace. Comme représenté sur l'image ci-dessous. Plus précisement, on représente ici l'ensemble des images qui peuvent correspondre à l'oeil à des images "de perroquets nettes".

<img src="figs/fig1.png" width="680" height="250">

- Les images observées/bruitées (dans notre cas par un bruit gaussien) sont des versions floues (gaussienne) des priors. Les densités d'observation associées sont une représetnation gaussienne des priors. Comme représenté sur l'image ci-dessous (pour différentes valeurs de la variance, de plus en plus grande). On note $y$ ces images et $p(y)$ leur probabilité associée.

<img src="figs/fig2.png" width="630" height="265">

## Principe de la méthode avec un bruit gaussien

### Lien entre $y$ et $x$ 

$y$ et $x$ sont liés par l'équation $y = x + z$, où $x \in \mathbb{R}^N$ représente l'image originale tirée de $p(x)$, et $z \sim \mathcal{N}(0, \sigma^2I_N)$ est un échantillon de bruit blanc gaussien.

### Approche d'apprentissage automatique classique pour le débruitage de l'image
Classiquement en apprentissage automatique, on utilise la méthode des moindres carrés.

Formule utilisée :
<img src="figs/fig3.png" width="630" height="265">

Schéma de la méthode :
![](https://latex.codecogs.com/svg.latex?\hat{x}(y)=min_{\hat{x}}\int||\hat{x}-x||^2p(x|y)dx=\int\left.xp(x|y)dx\right.) 

On obtient alors un réseau de neuronne (par exemple par CNN) entraîné pour le débruitage d'images bruité par un bruit gaussien noté $\hat{x}_{CNN}(y)$


### Approche d'apprentissage automatique utilisant le prior implicite 
 
Miyasawa (1961) a montré que l'estimation des moindres carrés pouvait être écrite (exactement) dans le cas d'une contamination gaussienne comme suit :

![](https://latex.codecogs.com/svg.latex?\hat{x}(y)=\int(xp(x|y)dx)=y+\sigma^2\nabla_y\log(p(y)))

L'approche va alors être de combiner ce résultat exact au réseau de neurones obtenu. **La méthode est expliquée ci-dessous dans la partie algorithme.** On obtient un algorithme simple qu'il suffit d'appliquer directement à une image bruitée.


### Avantage de la deuxième

Un algorithme d'apprentissage automatique classique réalise une minimisation de fonction entre les paramètres du réseau de neurones et l'image de départ. Cela mène généralement à un overfitting du problème. Notre réseau de neurone est efficace pour retrouver l'image de départ mais il n'est bon que pour cette image et il a eu besoin de cette image pour s'entraîner. Il demande également une longue phase d'entraînement.

Avec la méthode de l'article, pour retrouver une image proche de celle de l'image de départ, nous n'avons pas besoin de l'image de départ, il suffit juste d'appliquer l'algorithme. Du fait qu'il n'ait pas besoin de l'image de départ pour retrouver l'image cette méthode est universelle.

Les chercheurs ont montré de plus que l'image retrouvé est de meilleur qualité dans la deuxième méthode !

On a donc déjà trois avantages :
- Pas besoin de reconstruire un réseau neurones pour débruiter l'image et donc pas besoin de connaître l'image de départ (car plus besoin d'apprentissage supervisé).
- Méthode universelle à toute image ayant subi un filtre gaussien. Elle n'est pas efficacement spécifique à certianes images.
- Les résultats sont meilleurs qu'avec un CNN sans a priori.

### Exemple de débruitage par l'algorithme 
![](figs/synthesis_mnist_1.png)



# Partie plus technique sur les contributions de l'article: algorithmes en détail

## Algorithme de base

### Principe 
- On considère une condition initiale $y0$ (image bruitée par un bruit gaussien), on voudrait retrouver le $x$ (image réel), on note $\hat{x}(y0)$ l'image débruitée.
    Le bruit étant gaussien on connait l'expression exacte de cette image débruitée : ![](https://latex.codecogs.com/svg.latex?\hat{x}(y)=\int(xp(x|y)dx)=y+\sigma^2\nabla_y\log(p(y)))
    Mais nous ne connaissons pas la variance $\sigma$ qui a été utilisée pour bruiter l'image.
    
- En revanche, on connaît un CNN entraîné (pour le débruitage d'images bruité par un bruit gaussien) noté $\hat{x}_{CNN}(y)$ , c'est ici que nous allons utiliser la notion de prior implicite. Le réseau de neurones entraîné donnent des bons résultats. Il possède donc une version approchée de l'expression exacte que nous avons au-dessus. Nous allons donc le combiner à la formule pour obtenir une image débruitée de qualité.

- On a : $\sigma^2\nabla_y\log(p(y)))=\hat{x}(y)-y$. Ainsi, une version approchée de $\sigma^2\nabla_y\log(p(y0)))$ est donnée par : $\hat{x}_{CNN}(y0)-y0 = f(y0)$ ($f$ pour résiduel). Et donc $\hat{x}_{approx}(y0) = y0 + f(y0)$.

- En écrivant la valeur approchée de cette manière, on obtient juste $\hat{x}_{CNN}(y0)$ et alors on n'a pas exploité la formule. On ajoute donc un paramètre de contrôle $h$ compris entre 0 et 1 qui permet de contrôler Le résidu que l'on injecte à $y0$.
$\hat{x}_{approx}(y0) = y0 + h*f(y0)$

- $\hat{x}_{approx}(y0)= y0(1-h) + \hat{x}_{CNN}(y0)$ que l'on notera désormais $y1$ est donc une nouvelle image  bruitée de variance $\sigma1=\sigma0(1-h)$, expression bonne si l'on considère $\hat{x}_{CNN}$ parfait c'est à dire non stochastique. On remarque que $\sigma1<\sigma0$. Ainsi $y1$ est une image bruitée de l'espace des variétés mais moins bruité que celle d'avant.

- En réitérant ce procédé, on obtient au bout d'un certain nombre d'itération, une image non bruité de l'espace des variétés, c'est-à-dire une image réelle (ie non bruité, $p(x)$ dirac). 

- Cette image n'a pas de raison de correspondre à $\hat{x}(y0)$. Mais par la construction de l'algorithme, on obtient une image qui est **réelle / semble naturelle** et qui est **proche de $\hat{x}(y0)$** (car pas trop éloigné dans l'espace des variétés).

### Ecriture formelle

**Algorithm 0**: Coarse-to-fine gradient ascent method for sampling from the implicit prior of a denoiser, using denoiser residual $f(y) = \hat{x}(y) - y$.

*Parameters:* $\sigma_0, \sigma_L, h$

*Initialization:* $t = 1$, draw $y_0 \sim \mathcal{N}(0.5, \sigma_0^2I)$

**while** $\sigma_{t-1} \leq \sigma_L$ do:

- $d_t = f(y_{t-1})$
- $y_t \leftarrow y_{t-1} + h d_t$
- $t \leftarrow t+1$

**end while**


## Algorithme améliorée par ajour de bruit à chaque itération et modification du paramètre de contrôle

### Intérêt d'injecter un bruit
Deux raisons :
1. Éviter de tomber dans un minimum local
2. Permettre une exploration stochastique de l'ensemble de l'espace (manifold). L'image que l'on recherche étant la plus réelle, on a naturellement plus de chance de tomber proche de celle-ci si on réalise une grande exploration de l'espace.

### Intérêt de la modification du paramètre de contrôle

Les auteurs ont montré qu'un paramètre de contrôle constant amenait à des résultats de haute qualité, mais nécessitait un grand nombre d'itération. Pour accélérer la convergence on adpate le pas $h$ à chaque temps. On note ce nouveau $h$ : $h_t$;

*Remarque* : Une formule est proposée dans l'article, mais n'est pas justifié. Les auteurs affirme juste que cela améliore la convergence.


### Principe 

- En plus de l'algorithme de base, à chaque itération ils ajoutent un bout de bruit gaussien $\gamma_t z_t$ avec $z_t \sim \mathcal{N}(0, I)$ et $\gamma_t$ un paramètre de contrôle.

Comment choisir le paramètre de contrôle pour être sûre de converger ?
On souhaite avoir comme variance pour la nouvelle itération, une variance de la forme : $\sigma_t^2=\sigma_{t-1}^2(1-\beta h)^2$ avec $\beta$ entre 0 et 1.

En combinant les deux expressions, on obtient une formule pour $\gamma_t$.

- En plus de l'algorithme de base, à chaque itération le paramètre de contrôle de l'ajout de résidus est modifié d'une manière particulière : $h_t = \frac{h_0 t}{1 + h_0 (t - 1)}$. 


### Ecriture formelle

**Algorithm 1**: Coarse-to-fine stochastic ascent method for sampling from the implicit prior of a denoiser, using denoiser residual $f(y) = \hat{x}(y) - y$.

*Parameters:* $\sigma_0, \sigma_L, h_0, \beta $

*Initialization:* $t = 1$, draw $y_0 \sim \mathcal{N}(0.5, \sigma_0^2I)$

**while** $\sigma_{t-1} \leq \sigma_L$ do:

- $h_t = \frac{h_0 t}{1 + h_0 (t - 1)}$
- $d_t = f(y_{t-1})$
- $\sigma_t^2 = \|d_t\|^2/N$
- $\gamma_t^2 =((1 - \beta h_t)^2-(1 - h_t)^2) \sigma{t}^2$
- Draw $z_t \sim \mathcal{N}(0, I)$
- $y_t \leftarrow y_{t-1} + h_t d_t + \gamma_t z_t$
- $t \leftarrow t+1$

**end while**

### Remarque 

Si on prend un y quelconque, alors c'est algorithme permet de générer des images (image sythesis).

## Généralisation de l'algorithme a des problèmes d'inversion linéaire

### Nouveau problème 
On souhaite utiliser cette algorithme dans le cas ou $x$ est incomplet. Nous pouvons par exemple citer les problèmes :
- suppression d'un bloc de pixels
- suppression de pixel aléatoires
- image compressée, c'est-à-dire qui a perdu en résolution
- image dont il manque des fréquences

Tous ces problèmes sont dit linéaires car nous pouvons écrire l'image incomplète de $x$ noté $x^c$ comme le produit de $x$ par une matrice de transformation : $x^c = M^Tx$. Ces notations sont expliquées plus en détails dans le paragraphe ci-dessous.

À retenir, pour ce nouveau problème :

**Nous n'avons plus $y$ une image bruitée de l'image $x$, mais nous avons $y$ l'image bruitée d'une image incomplète de $x$. On ne s'intéresse donc plus à $p(y|x)$ mais $p(y|x^c)$ avec $x^c = M^Tx$ une transformation linéaire de $x$.**




### Rappel d'algèbre linéaire et notation pour la suite
Il y a deux manières de décomposer un élément $x$ d'un ensemble $E$ : 

#### 1. Décompositon orthogonale 
Si $E$ est un espace vectoriel avec un produit scalaire, on peut effectuer une décomposition de tout élément $x$ de $E$ en utilisant un espace de projection $P$ et son complémentaire orthogonal $P^\perp$.

Soit $E$ un espace vectoriel euclidien et $P$ un sous-espace de $E$. Alors, pour tout $x \in E$, il existe un unique $p_x \in P$ et un unique $p_x^\perp \in P^\perp$ tels que $x = p_x + p_x^\perp$, où $p_x$ est dans $P$ et $p_x^\perp$ est dans $P^\perp$, et ils sont orthogonaux l'un à l'autre.

Mathématiquement, cela peut être exprimé comme suit, en supposant que $\langle \cdot, \cdot \rangle$ représente le produit scalaire :

$x = p_x + p_x^\perp, \text{ où } p_x \in P, \ p_x^\perp \in P^\perp, \ \text{et} \ \langle p_x, p_x^\perp \rangle = 0.$

Dans ce contexte, la décomposition est unique et dépend uniquement du choix de $P$.

**Dans cette décomposition, les projections sont des vecteurs de $E$.**

#### 2. Décompositon orthogonale matricielle projeté 
Dans notre cas $E=\mathbb{R}^n$ avec $n$ le nombre de pixels. On peut donc écrire le problème matriciellemeent.

Soit $x$ un élément de $E$, $P\in\mathbb{R}^m$ et $P^\perp\in\mathbb{R}^k$ deux sous-espaces complémentaires de dimension inférieure à $n$, tels que $P \oplus P^\perp = E$. La décomposition matricielle peut être réduite à l'équation suivante :

$x = Mx^c + \bar{M}x^u$ avec :
- $M\in\mathbb{R}^{n \times m}$
- $\bar{M}\in\mathbb{R}^{n \times k}$.
- $x^c=M^Tx$
- $x^u=\bar{M}^Tx$

#### Lien entre les décomposition
C'est décomposition sont équivalente et on a :
- $p_x = Mx^c = MM^Tx$
- $p_x^\perp = \bar{M}x^u = \bar{M}\bar{M}^Tx = (I_n - MM^T)x$

### Comment déterminer $p(y|x^c)$ ? 

- Notons tout d'abord que $y$ peut s'écrire : $y=Mx^c+z$ avec $z$ un bruit gaussien comme définit auparavant.

- Pour utiliser le principe de l'algorithme de base, nous avons besoin d'exprimer le terme : $\sigma^2\nabla_y\log(p(y|x^c)))$

On décompose $y$ comme on a décomposé $x$ sur la base définie par la matrice $M$ et son complémentaire orthogonal.

$p(y|x^c) = p(y^c, y^u|x^c) = p(y^u|y^c, x^c)p(y^c|x^c) = p(y^u|x^c)p(y^c|x^c)$

Et donc : 

$\sigma^2\nabla_y\log(p(y|x^c))) = \sigma^2\nabla_y\log(p(y^c|x^c))) + \sigma^2\nabla_y\log(p(yû|x^c)))$

C'est écriture de $A = \sigma^2\nabla_y\log(p(y|x^c)))$ est en fait équivalente à une décomposition sur la base définie par la matrice $M$ et son complémentaire orthogonal :

$A = p_A + p_A^\perp$

- Déterminons chaque composante (pour faciliter la compréhesion, il est une bonne idée de se représenter le problème sous forme de dessin dans le cas où $M$ correspond à la suppression d'un bloc de pixel) :

### $p_A$
 

- **Si** on avait $y=x+z$ alors par Miyasawa, on aurait :

$B = \sigma^2\nabla_y\log(p(y))) = \hat{x}(y)-y = p_{x-y}+p_{x-y}^\perp$

Par unicité de la décompostion : $p_B = p_{x-y}$

Or : $p_{x-y} = M(x^c - y^c)$

Comme on a $y^c=x^c + z^c$, on est donc exactement dans le cadre de Miyasawa et donc $p_B = p_A$ ie $p_A = M(x^c - y^c)$. 

Pour résumer, comme dans le sous-espace définie par la matrice $M$, on a $y^c=x^c + z^c$ $ie$, comme on est dans le cadre de Miyasawa pour $y^c$, on a exactement : $p_A = M(x^c - y^c)$. 


### $p_A^\perp$

- Pour l'espace complémentaire orthogonale, on a : $y^u = z^u$ donc on ne peut pas réaliser le même raisonnement. On va être obligé de réaliser la même manœuvre que l'on avait utilisé pour l'algorithme de base, c'est-à-dire d'utiliser la version approchée du gradient de probabilité avec le réseau de neurones entraîné.

On a : $\hat{x}(y) \simeq y + A$ *ie* $A\simeq f(y)$ le résidu de la valeur déterminée par le réseau de neuronne.

Par unicité de la décomposition (cf rappel d'algèbre), on a : $p_A^\perp=(I_n - MM^T)A$.

En combinant les deux dernières égalité, on a donc : $p_A^\perp=(I_n - MM^T)f(y)$

###  Résulat total

- **$ \nabla \log p(y|x^c) = (I_n - MM^T)f(y) + M(x^c - M^Ty). $** : Expression utilisant que les paramètres connus !

- Par rapport à l'algorithme de base, nous remplaçons donc : $d_t = f(y_{t-1})$ par $d_t = (I-M M^T)f(y_{t-1}) + (Mx^c - MM^Ty_{t-1})$

### Ecriture formelle



**Algorithm 2**: Coarse-to-fine stochastic ascent method for sampling from sampling from $p(x|M^Tx=x^c)$, based on the residual of a denoiser, $f(y) = \hat{x}(y) - y$. Note : $e$ is an image of ones.

*Parameters:* $\sigma_0, \sigma_L, h_0, \beta, M, x^c $

*Initialization:* $t = 1$, draw $y_0 \sim \mathcal{N}(0.5(I-M M^T)e+ Mx^c, \sigma_0^2I)$

**while** $\sigma_{t-1} \leq \sigma_L$ do:

- $h_t = \frac{h_0 t}{1 + h_0 (t - 1)}$
- $d_t = (I-M M^T)f(y_{t-1}) + (Mx^c - MM^Ty_{t-1})$
- $\sigma_t^2 = \|d_t\|^2/N$
- $\gamma_t^2 =((1 - \beta h_t)^2-(1 - h_t)^2) \sigma{t}^2$
- Draw $z_t \sim \mathcal{N}(0, I)$
- $y_t \leftarrow y_{t-1} + h_t d_t + \gamma_t z_t$
- $t \leftarrow t+1$

**end while**





## Quelques résultats de l'article par l'agorithme final
### Résolution de problèmes inverses linéaires en utilisant le prior implicite


#### 1. Impaint (Restauration)
Ici, l'algorithme doit remplir un bloc de pixels manquants (c'est-à-dire, la mesure consiste en les pixels à l'extérieur du bloc manquant). Avec différentes initialisations et différents bruits additifs, l'algorithme arrive à différentes solutions (différents points sur la variété). 

![](figs/inpaint_samples_flower.png)


#### 2. Pixels manquants aléatoires
Les mesures consistent en un sous-ensemble de 10 % des pixels sélectionnés au hasard.


![](figs/random_pix_samples_zebra.png)


#### 3. Super-résolution
Les mesures sont des moyennes de chaque bloc (non chevauchant) de 4x4 pixels.


![](figs/super_res_pepper.png)


#### 4. Super résolution spectrale
Les mesures consistent en 10 % des coefficients de Fourier (les fréquences les plus basses).


![](figs/deblur_butterfly.png)


#### 5. Base aléatoire (compressive sensing)
Ici, la matrice de mesure $M$ contient des vecteurs unitaires orthogonaux aléatoires qui englobent un sous-espace dont la dimensionalité est de 10 % de la dimensionalité de l'image. Ainsi, les mesures sont les projections de l'image sur un sous-espace aléatoire de basse dimension.


![](figs/rand_basis_soldier.png)


# Spécificités de l'article par rapport à l'état de l'art

## Etat de l'art 

1942, Weiner : débruitage d'un bruit gaussien équivalent à une projection sur les basses fréquences.

1994, Donoho & Johnstone : généralisation de ce résultat, le bruitage d'une image correspond à la projection sur un sous espace adapté (les basses fréquences par exemple pour le bruit gaussien).

2017, Ehangetal : DnCNN, architecture de réseau de neurones spécialisée dans le débruitage d'images, exploitant la puissance des CNN pour apprendre des modèles de débruitage à partir de données d'entraînement.

La question des auteurs est, qu'est-ce que l'on fait une fois que l'on a obtenu ce réseau de neurones optimisé avec le DnCNN.
- Analyse : Propriété de la carte prise.
- Interprétation : Quel est l'implicite "prior".
- Généralisation : Peut-on l'utiliser pour autre chose ?

# Notebook permettant de reproduire une partie des expériences

# Présentation des résultats reproduits

Bilbio :
    COnférence : Eero Simoncelli - Making use of the Prior Implicit in a Denoiser
    https://www.youtube.com/watch?v=bquD_tSeQlc